In [ ]:
from pathlib import Path

In [ ]:
warehouse_path = str(Path.home() / "temp" / "iceberg" / "spark-warehouse" / "local")
catalog_name = "local"

In [ ]:
from sedona.spark import *

config = (
    SedonaContext.builder()
    .config(
        "spark.jars.packages",
        "org.apache.sedona:sedona-spark-3.5_2.12:1.7.1,"
        "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.0,"
        "org.datasyslab:geotools-wrapper:1.7.1-28.5"
    )
    .config(
        "spark.jars.repositories",
        "https://artifacts.unidata.ucar.edu/repository/unidata-all",
    )
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
    )
    .config(
        f"spark.sql.catalog.{catalog_name}",
        "org.apache.iceberg.spark.SparkCatalog"
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.type", "hadoop"
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.warehouse",
        f"{warehouse_path}/{catalog_name}"
    )
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.sql.session.timeZone", "UTC")
    .getOrCreate()
)
sedona = SedonaContext.create(config)

In [ ]:
sedona.sql("""
DROP TABLE IF EXISTS local.db.locations;
""")

In [ ]:
sedona.sql("""
    CREATE TABLE local.db.locations (
        id STRING,
        name STRING,
        geom GEOMETRY
    )
    USING iceberg
    TBLPROPERTIES ('format-version'='3');
""")

In [ ]:
sedona.stop()

In [1]:
import teehr